# Imports

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import matplotlib
import matplotlib.pyplot as plt
from tcpyPI import pi, cape
import datetime
import cftime
import dask
import dask.array as da
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from tqdm import tqdm, trange
import time
import scipy
import xesmf as xe

# Functions

In [ ]:
def calcGP(vor, H, V_pot, V_shear):
    # absolute vorticity at 850hPa, RH at 700hPa, Potential Intensity, Vertical wind shear 850hPa and 200hPa
    I = (np.sqrt(np.abs(10 ** 5 * vor)) ** 3) * ((H / 50) ** 3) * ((V_pot / 70) ** 3) * ((1 + 0.1 * V_shear) ** (-2))
    GP = I.compute()

# Data load and adjustments

In [ ]:
u = xr.open_dataset('Data/JRA55_033_ugrd.nc', chunks={'time': 12}).astype('float64')
v = xr.open_dataset('Data/JRA55_034_vgrd.nc', chunks={'time': 12}).astype('float64')
rv = xr.open_dataset('Data/JRA55_043_relv.nc', chunks={'time': 12}).astype('float64')
rh = xr.open_dataset('Data/JRA55_052_rh.nc', chunks={'time': 12}).astype('float64')
# tmp = xr.open_dataset('Data/JRA55_011_tmp.nc', chunks={'time': 12}).astype('float64')
# mix = xr.open_dataset('Data/JRA55_051_spfh.nc', chunks={'time': 12}).astype('float64')
# mslp = xr.open_dataset('Data/JRA55_002_prmsl.nc', chunks={'time': 12}).astype('float64')
# sst = xr.open_dataset('Data/hadisst.187001-201912.nc',chunks={'time': 12}).astype('float64')

In [ ]:
# # adjust time objects in hadisst
# sst['time'] = pd.to_datetime(sst['time'], yearfirst=True, format='%Y%m')

In [ ]:
# regrid settings
grid = 1.25
lons = np.arange(0, 360+grid, grid)
lats = np.arange(-90, 90 + grid, grid)

grid_in = {"lon": tmp.longitude.values, "lat": tmp.latitude.values}
grid_out = {"lon" : lons, "lat": lats}
grid2 = {"lon": sst.lon.values, "lat": sst.lat.values}

re_1to125 = xe.Regridder(grid2, grid_out, "bilinear", periodic=True) # Hadisst
regridder = xe.Regridder(grid_in, grid_out, "bilinear", periodic=True) 
regridder, re_1to125  # print basic regridder information.

In [ ]:
# regrid with dask
u = regridder(u).compute()
v = regridder(v).compute()
rv = regridder(rv).compute()
rh = regridder(rh).compute()
# tmp = regridder(tmp).compute()
# mix = regridder(mix).compute()
# mslp = regridder(mslp).compute()
# sst = re_1to125(sst).compute()

In [ ]:
# Select coordinates of interest
box = [-100, 20, -20, 40]
rv_box = ATLN_box(rv, box).compute()
rh_box = ATLN_box(rh, box).compute()
u_box = ATLN_box(u, box).compute()
v_box = ATLN_box(v, box).compute()
# tmp_box = ATLN_box(tmp, box).compute()
# mix_box = ATLN_box(mix, box).compute()
# mslp_box = ATLN_box(mslp, box).compute()
# sst_box = ATLN_box(sst, box).compute()

In [ ]:
# define pressure levels
pressure_box = u_box['isobaricInhPa']
u850 = u_box.sel(isobaricInhPa=850)
v850 = v_box.sel(isobaricInhPa=850)
u200 = u_box.sel(isobaricInhPa=200)
v200 = v_box.sel(isobaricInhPa=200)
rh700 = rh_box.sel(isobaricInhPa=700)
rv850 = rv_box.sel(isobaricInhPa=850)

In [ ]:
# remove unused variables
del u, v, rv, rh, rv_box, rh_box, u_box, v_box

# Climatologies

# Calculate